## Imports libraries

In [2]:
import pandas as pd
import numpy as np
import mne

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.use('Qt5Agg')

## Opens and cleans the text file

In [3]:
def clean_text(file, nrow):
    with open(file, "r") as file:

        # Creates the placeholder data object
        data = np.zeros((3157171,2))

        i = 0
        for line in file:
            
            if i == nrow:
                break
            
            # Creates a row out of the line and adds it to the data object
            row = line.split("\t")
            row[2] = row[2].replace("\n", "")
            row = np.array([[float(row[0]), float(row[2]) * (10 ** -6)]])
            data[i] = row
            
            i += 1

    # Removes the placeholder row at the beginning
    data = data[1:]
    return data

In [4]:
data = clean_text(file = "exp1.txt", nrow = -1)
data[:,1] = data[:,1] + 26e-6
data[:,0] = data[:, 0] - 98.9975

df = pd.DataFrame(data=data)
df.to_csv("exp1_cleaned.csv", index=False)

## Convert data to mne

In [5]:
def create_mne(data, description = "Data from the stretch receptor lab"):

    # Calculates the sampling frequency
    sam_freq = 1 / (data[1,0] - data[0,0])
    sam_freq = 4000

    # Creates the mne info object
    info = mne.create_info(["Channel 1"], sfreq = sam_freq, ch_types = ["ecog"])
    info["description"] = description

    # Creates the mne raw object
    raw_data = np.array([data[:, 1]])
    mne_raw = mne.io.RawArray(data = raw_data, info = info)

    return mne_raw


raw = create_mne(data)
raw.plot()

Creating RawArray with float64 data, n_channels=1, n_times=3157170
    Range : 0 ... 3157169 =      0.000 ...   789.292 secs
Ready.
Using matplotlib as 2D backend.


<MNEBrowseFigure size 2560x1334 with 4 Axes>

In [6]:
def detect_spikes(data, threshold, sfreq):
    """
        data: The time series data with the format: [[timestamp, amplitude],...]
        threshold: voltage threshold for spike detection (in microvolts)
        sfreq: The sampling frequency (in Hz)

        returns spike_data. Format: [Timestamp, width, amplitude]
    """
    threshold = threshold * 10**-6
    spike_data = []
    check_falling = lambda data, i: (data[i + 1, 1] - data[i, 1]) < 0
    check_rising = lambda data, i: data[i + 1, 1] - data[i, 1] > 0
    

    spike_detected = False
    for i in range(0, len(data) - 1):
        
        if data[i, 1] < threshold and check_falling(data, i) and not spike_detected:
            spike_detected = True
            
        if spike_detected and check_rising(data, i):
            spike_detected = False
            spike_data.append(data[i])
            
    
    return np.array(spike_data)

spikes = detect_spikes(data, -18, 4000)
spikes




array([[ 2.5875000e-01, -2.3093750e-05],
       [ 5.6575000e-01, -2.3031250e-05],
       [ 8.6100000e-01, -2.5812500e-05],
       ...,
       [ 7.8913725e+02, -5.0406250e-05],
       [ 7.8915950e+02, -4.9968750e-05],
       [ 7.8918150e+02, -4.9000000e-05]])

## Calculate instantanous frequencies

In [7]:
def inst_freq(spike_data):

    inst_freqs = np.zeros((len(spike_data) - 1, 2))

    for i in range(len(spikes) - 1):
        inst_freqs[i] = [spike_data[i, 0], 1 / (spike_data[i + 1, 0] - spike_data[i, 0])]
    
    
    return inst_freqs

ifd = inst_freq(spike_data=spikes)
ifd[:, 0] = ifd[:, 0]


x = ifd[:, 0]
y = ifd[:, 1]





## Get stretch adaptation data

In [8]:
def get_freq_data(file, n):
    with open(file, "r") as file:
        freq_data = np.zeros((n, 2)) 
        
        i = 0
        for line in file:
            line = line.split(",")
            freq_data[i] = [float(line[0]), float(line[1])]
            i += 1

    return freq_data

def fit_function(R_inf, R0, tau, x_range = (0, 25), n = 10000):
    x = np.linspace(start = x_range[0], stop = x_range[1], num = n)
    y = R_inf + R0 * np.exp(-x / tau)

    data = np.zeros((n,2))
    data[:,0] = x
    data[:, 1] = y
    print(data)
    return data

s15mm = get_freq_data("exp1/15mm_fit_data.txt", 174) # 28.1s
s15mm[:,0] = s15mm[:,0] - s15mm[0,0]
s20mm = get_freq_data("exp1/20mm_fit_data.txt", 419) # 34.2s
s20mm[:,0] = s20mm[:,0] - s20mm[0,0]
s25mm = get_freq_data("exp1/25mm_fit_data.txt", 599) # 53.16s
s25mm[:,0] = s25mm[:,0] - s25mm[0,0]

data15mm = fit_function(R_inf = 6.78, R0 = 1.53, tau = 2.13)
data20mm = fit_function(R_inf = 16.0, R0 = 3.91, tau = 4.98)
data25mm = fit_function(R_inf = 22.1, R0 = 7.30, tau = 6.53)


[[0.00000000e+00 8.31000000e+00]
 [2.50025003e-03 8.30820510e+00]
 [5.00050005e-03 8.30641230e+00]
 ...
 [2.49949995e+01 6.78001226e+00]
 [2.49974997e+01 6.78001224e+00]
 [2.50000000e+01 6.78001223e+00]]
[[0.00000000e+00 1.99100000e+01]
 [2.50025003e-03 1.99080374e+01]
 [5.00050005e-03 1.99060759e+01]
 ...
 [2.49949995e+01 1.60258476e+01]
 [2.49974997e+01 1.60258346e+01]
 [2.50000000e+01 1.60258216e+01]]
[[0.00000000e+00 2.94000000e+01]
 [2.50025003e-03 2.93972055e+01]
 [5.00050005e-03 2.93944120e+01]
 ...
 [2.49949995e+01 2.22588422e+01]
 [2.49974997e+01 2.22587814e+01]
 [2.50000000e+01 2.22587206e+01]]


In [13]:
plt.scatter(s15mm[:,0],s15mm[:,1], color = "blue")
plt.scatter(s20mm[:,0],s20mm[:,1], color = "red")
plt.scatter(s25mm[:,0],s25mm[:,1], color = "green")

plt.plot(data15mm[:,0], data15mm[:,1], linewidth = 3, color = "black")
plt.plot(data20mm[:,0], data20mm[:,1],linewidth = 3, color = "black")
plt.plot(data25mm[:,0], data25mm[:,1], linewidth = 3, color = "black")

plt.title(label=" ", fontsize = 30, pad = 20)

plt.xticks(fontsize = 17)
plt.yticks(fontsize = 17)

plt.xlabel(xlabel="Time (s)", fontsize = 20,)
plt.ylabel(ylabel="Action potential firing rate (Hz)", fontsize = 20)
plt.xlim((0, 26))

plt.show()